In [4]:
from src.sentiment_analysis import *
test_examples, test_ids = nusax_sentiment()

from collections import defaultdict
# get_gpt_generation() function from utils
from utils import * 
import jsonlines

# Sentiment Analysis

## Prompting

In [8]:
def sentiment_analysis():
    answers = defaultdict(list)
    do_print = True
    
    for ex in test_examples:
        prompt = f"\"{ex['text']}\". What would be the sentiment of the text? positive, negative, or neutral?"
    
        ###### HERE YOU CAN CHANGE IT TO YOUR OWN MODELS########
        gen = get_gpt_generation(prompt)
        ########################################################
    
        # Here, you cannot fully depedent on parsed answer --> highly recommend double check
        gen_split = gen.lower().split()
    
        a = -1
        for x in gen_split:
            if 'positive' in x:
                a = 'positive'
            elif 'negative' in x:
                a = 'negative'
            elif 'neutral' in x:
                a = 'neutral'
                
            if a != -1:
                break
                
        if a == -1:
            a = 'x' 

        parsed_answer = a
            
        ex['gen'] = gen
        ex['parsed_answer'] = parsed_answer
        answers[ex['lang']].append({'label': ex['label'], 'a' : parsed_answer, 'gen': gen})
        
        with jsonlines.open('results/sentiment_analysis.jsonl', mode='a') as writer:
            writer.write(ex)
    
        if do_print:
            print(f"{ex['lang']})) {ex['label']} | {parsed_answer} | {gen}")


In [ ]:
sentiment_analysis()

## Scoring


In [27]:
objs= []

with jsonlines.open('results/sentiment_analysis.jsonl') as read_file:
    for line in read_file.iter():
    	objs.append(line)

do_print = False

counter = defaultdict(list)
for o in objs:
    a = o['a']

    correct = 1 if a == o['label'] else 0
    counter[o['lang']].append(correct)
    
    if do_print:
        print(f"{o['lang']} | {o['label']} | {a} | {gen.strip()} | {correct}")
    

In [28]:
# # PRINT
for k, v in counter.items():
    print(k,": ", sum(v)/len(v))


Buginese :  0.08
English :  0.82
Indonesian :  0.76
Javanese :  0.6


# LID


In [ ]:
import pandas as pd

## Prompting

In [39]:
def lid():
    df = pd.read_csv("data/LID.csv")
    for idx, row in df.iterrows():
        prompt = row['Text']
        label = row['label']
        
    ###### HERE YOU CAN CHANGE IT TO YOUR OWN MODELS########
        gen = get_gpt_generation(prompt)
    ########################################################
        
        if label in gen.lower():
            a = 1
        else:
            a = 0
    
        obj = {
            'prompt' : prompt,
            'label' : label, 
            'gen' : gen,
            'correct' : a
        }
        with jsonlines.open('lid.jsonl', mode='a') as writer:
            writer.write(obj) 


In [ ]:
lid()

## Scoring

In [24]:
from collections import defaultdict


lid_objs= []
with jsonlines.open('results/lid.jsonl') as read_file:
    for line in read_file.iter():
    	lid_objs.append(line)

# COUNT
do_print = False
counter = defaultdict(list)
for o in lid_objs:
    counter[o['label']].append(o['correct'])

    if do_print:
        if o['correct'] == 0:
            print(o['label'], "|| ", o['gen'], o['prompt'])

# PRINT
for k, v in counter.items():
    print(k,": ", sum(v)/len(v))


indonesian :  1.0
english :  0.92
javanese :  0.9
buginese :  0.64


## SA - F1 SCORE

In [25]:
import pandas as pd
from collections import defaultdict
df = pd.read_csv("sa.csv")

result_lang_label = defaultdict(list)
result_lang_gen = defaultdict(list)
for idx, row in df.iterrows():
    result_lang_label[row['lang']].append(row['label'])
    result_lang_gen[row['lang']].append(row['gen'])

from sklearn.metrics import f1_score
lang = 'Indonesian'
f1_score(result_lang_label[lang], result_lang_gen[lang], average='macro')

0.8955608834641092

buginese 0.12373737373737373

English 0.8121693121693121

JAVANESE 0.6167613636363637

Indonesian 0.8955608834641092